### Audio

In [90]:
from joblib import load
import librosa
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib 


In [91]:
# Load the best model from the file
audio_loaded_model = load('Final_Audio_Model.joblib')


In [92]:
# Extract more audio features
def extract_features(file_path):
    try:
        y, sr = librosa.load(file_path)
    except Exception as e:
        print(f"Error loading file: {file_path} - {str(e)}")
        return None
    
    if file_path.lower().endswith(".wav"):
        features = [
            np.mean(librosa.feature.chroma_stft(y=y, sr=sr)),
            np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)),
            np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr)),
            np.mean(librosa.feature.mfcc(y=y, sr=sr)),
            np.mean(librosa.feature.zero_crossing_rate(y)),
            np.mean(librosa.feature.spectral_contrast(y=y, sr=sr)),
            np.mean(librosa.feature.tempogram(y=y, sr=sr))
        ]
        return features
    else:
        return None

In [93]:
scaler = joblib.load('audio_scaler.save')
label_encoder=joblib.load("audio_encoder.save")

In [94]:
# Initialize global variables
predicted_emotions = []
probabilities = []

def predict_emotions(audio_path, audio_loaded_model, label_encoder):
    global predicted_emotions, probabilities  # Declare them as global
    
    try:
        y, sr = librosa.load(audio_path)
    except Exception as e:
        print(f"Error loading audio: {str(e)}")
        return None

    # Extract audio features from the input audio
    features = extract_features(audio_path)
    if features is not None:
        # Standardize the features
        features = scaler.transform([features])
        
        # Predict the emotion probabilities using the loaded model
        emotion_probabilities = audio_loaded_model.predict_proba(features)
        
        # Get the top 3 emotions with the highest probabilities
        top_emotions_indices = (-emotion_probabilities).argsort()[0][:3]
        
        predicted_emotions = label_encoder.inverse_transform(top_emotions_indices)
        probabilities = [emotion_probabilities[0][index] for index in top_emotions_indices]
        
        return predicted_emotions, probabilities
    else:
        return None

# Provide an audio path to predict emotions
audio_path = "output_audio.wav"
predict_emotions(audio_path, audio_loaded_model, label_encoder)

if predicted_emotions is not None:
    print("Predicted Emotions:")
    for emotion, probability in zip(predicted_emotions, probabilities):
        print(f"{emotion}: {probability:.2f}")
else:
    print("No emotions predicted.")


Predicted Emotions:
Fear: 0.44
Happy: 0.18
Angry: 0.17


## Image

In [97]:
import numpy as np
from PIL import Image
from keras.models import load_model
import os


In [98]:
# Load the saved model
image_loaded_model = load_model('Final_Image_Model.h5')

In [99]:
# Define the class names for your dataset
class_names = ['anger', 'disgust', 'fear', 'happy', 'neutral', 'sad']


In [100]:
import os
import numpy as np
from PIL import Image

# Assume you have a pre-trained model loaded as 'image_loaded_model'
# and a list of class names loaded as 'class_names'
top3_emotions=[]
top3_probabilities=[]
def load_and_preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((80, 80))
    img = img.convert('RGB')
    img_array = np.array(img) / 255.0
    img_array = img_array.reshape(1, 80, 80, 3)
    return img_array

def classify_frames(frames_folder, model, class_names):
    global top3_emotions, top3_probabilities  # Declare them as global
    emotion_counts = {emotion: 0 for emotion in class_names}
    total_frames = 0

    for frame_file in os.listdir(frames_folder):
        frame_path = os.path.join(frames_folder, frame_file)

        # Load and preprocess each frame
        frame_data = load_and_preprocess_image(frame_path)

        # Make predictions on the frame
        predictions = model.predict(frame_data)

        # Get the top predicted class label and confidence score
        top_class_index = np.argmax(predictions[0])
        top_class = class_names[top_class_index]
        confidence = predictions[0][top_class_index]

        # Update counts
        emotion_counts[top_class] += confidence
        total_frames += 1

    # Calculate probabilities for each emotion
    emotion_probabilities = {emotion: count / total_frames for emotion, count in emotion_counts.items()}

    # Get the top 3 emotions and their probabilities
    top3_emotions = sorted(emotion_probabilities, key=emotion_probabilities.get, reverse=True)[:3]
    top3_probabilities = [emotion_probabilities[emotion] for emotion in top3_emotions]

    # Print the final top 3 emotions and their probabilities
    print('Final Top 3 Emotions:')
    for emotion, probability in zip(top3_emotions, top3_probabilities):
        print(f'{emotion}: {probability:.2f}')

if __name__ == "__main__":
    frames_folder = "frames_folder"  # Update with the correct path to your frames folder

    classify_frames(frames_folder, image_loaded_model, class_names)


1/1 [==============================] - 0s 44ms/step
Final Top 3 Emotions:
neutral: 0.29
fear: 0.19
happy: 0.05


## Text

In [101]:
import joblib
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
import numpy as np

In [102]:
# Load the saved model
text_loaded_model = joblib.load('Final_Text_Model.pkl')

In [103]:
tokenizer = joblib.load('text_tokenizer.save')

In [104]:
max_sequence_length = 150

In [105]:
# Load the text data from the saved transcript file
transcript_file = "transcript.txt"
with open(transcript_file, 'r') as file:
    text_data = file.read()

# Tokenize and pad the text data
text_seq = tokenizer.texts_to_sequences([text_data])
text_pad = pad_sequences(text_seq, maxlen=max_sequence_length, padding='post')

# Assuming text_loaded_model is an ExtraTreesClassifier with 50 features
# Make sure text_pad has the correct shape with 50 features
text_pad_50_features = text_pad[:, :50]

# Make a prediction using the loaded model
predicted_probabilities = text_loaded_model.predict_proba(text_pad_50_features)

# Get the top three emotions and their probabilities
top_emotions_indices = np.argsort(predicted_probabilities[0])[-3:][::-1]
top_emotions = label_encoder.inverse_transform(top_emotions_indices)
top_probabilities = predicted_probabilities[0][top_emotions_indices]

# Print the top three predicted emotions and their probabilities
print('Top 3 Predicted Emotions:')
for emotion, probability in zip(top_emotions, top_probabilities):
    print(f"{emotion}: {probability:.4f}")


Top 3 Predicted Emotions:
Happy: 0.4283
Sad: 0.1695
Neutral: 0.1657


### Final Results

In [107]:
top_probabilities = top_probabilities.tolist()
predicted_emotions = predicted_emotions.tolist()

In [117]:
top_emotions=top_emotions.tolist()

In [118]:
# Combine the results from different models
all_emotions = predicted_emotions+ top3_emotions + top_emotions
all_probabilities = probabilities + top3_probabilities + top_probabilities

# Create a dictionary to associate emotions with their probabilities
emotion_prob_dict = dict(zip(all_emotions, all_probabilities))

# Sort the dictionary by probabilities in descending order
sorted_emotion_prob = {k: v for k, v in sorted(emotion_prob_dict.items(), key=lambda item: item[1], reverse=True)}

# Extract the top N emotions and their probabilities
final_top_emotions = list(sorted_emotion_prob.keys())[:3]
final_top_probabilities = list(sorted_emotion_prob.values())[:3]

# Print the final results
print("Final Top Emotions:", final_top_emotions)
print("Final Top Probabilities:", final_top_probabilities)


Final Top Emotions: ['Fear', 'Happy', 'neutral']
Final Top Probabilities: [0.44, 0.42825, 0.2883080468474034]
